In [4]:
import pickle 
import numpy as np
import pandas as pd
import statistics
from PIL import Image
import os 
import random
import time
import matplotlib.pyplot as plt
import netaddr

In [5]:
import dgl
import torch
import torch.nn as nn
import dgl.function as fn
import dgl.nn as dglnn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [10]:
#pip install netaddr

In [13]:
#directory should change based on the device, for laptop it should be users//jalal for office desktop it is users//jghadermazi

In [6]:
wed_data = pd.read_pickle(r'C:\\Users\\jghadermazi\\Box\\GAN Project\\Sample data\\Friday_Packet_Level_Data.pkl')

In [36]:
df1 = wed_data.copy()

In [37]:
df1[['Payload']] = df1[['Payload']].replace('', '00')

In [38]:
df1

,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
0,1499432653.990571000,192.168.10.9,205.174.165.73,1841,8080,13099,0,00c1b114eb31b8ac6f1d1f6c0800,450000346c8f40008006508bc0a80a09cdaea549,07311f905c691851000000008002200075eb0000020405...,00,fwd,1.0,12.0
1,1499432653.991397000,205.174.165.73,192.168.10.9,8080,1841,13099,0,b8ac6f1d1f6c00c1b114eb310800,45000034000040003e06ff1acdaea549c0a80a09,1f90073113dd44cb5c69185280127210cb220000020405...,00,bwd,1.0,12.0
2,1499432653.991520000,192.168.10.9,205.174.165.73,1841,8080,13099,1,00c1b114eb31b8ac6f1d1f6c0800,450000286c90400080065096c0a80a09cdaea549,07311f905c69185213dd44cc501001007d050000,000000000000,fwd,1.0,12.0
3,1499432653.992542000,192.168.10.9,205.174.165.73,1841,8080,13099,1,00c1b114eb31b8ac6f1d1f6c0800,450000ea6c91400080064fd3c0a80a09cdaea549,07311f905c69185213dd44cc50180100307c0000,474554202f6170692f706f703f626f7469643d6d697461...,fwd,1.0,12.0
4,1499432653.993057000,205.174.165.73,192.168.10.9,8080,1841,13099,1,b8ac6f1d1f6c00c1b114eb310800,45000028822a40003e067cfccdaea549c0a80a09,1f90073113dd44cc5c691914501000ed7c560000,000000000000,bwd,1.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5260217,1499454300.212460000,172.217.12.205,192.168.10.14,443,58093,280715,3982,b8ac6f3607ee00c1b114eb310800,450000f7dfe60000340621beacd90ccdc0a80a0e,01bbe2eda84cbf47d787ec2350180169decb0000,17030300ca0000000000000003b9423a0b5901a1d34005...,bwd,0.0,0.0
5260218,1499454300.212688000,172.217.12.205,192.168.10.14,443,58093,280715,4189,b8ac6f3607ee00c1b114eb310800,4500019ddfe7000034062117acd90ccdc0a80a0e,01bbe2eda84cc016d787ec2350180169e0870000,1703030170000000000000000427cfa8c91d47cda4d459...,bwd,0.0,0.0
5260219,1499454300.212736000,172.217.12.205,192.168.10.14,443,58093,280715,4562,b8ac6f3607ee00c1b114eb310800,450000acdfe8000034062207acd90ccdc0a80a0e,01bbe2eda84cc18bd787ec235018016903ef0000,170303007f000000000000000529465b5b09e2c6a7bf19...,bwd,0.0,0.0
5260220,1499454300.212738000,172.217.12.205,192.168.10.14,443,58093,280715,4694,b8ac6f3607ee00c1b114eb310800,45000056dfe900003406225cacd90ccdc0a80a0e,01bbe2eda84cc20fd787ec2350180169e55a0000,170303002900000000000000066521f26029ac678a3244...,bwd,0.0,0.0


In [39]:
#benign    0
#dos slowloris    1
#dos slowhttptest   2
#dos hulk     3
#dos goldeneye    4
#heartbleed    5
#ftp patator    6
#ssh patator    7
#webattack bruteforce    8
#webattack xss     9
#webattack sql injection 10
#infiltration     11
#botnet     12
#portscan   13
#ddos     14

In [40]:
df1['Class Label'].value_counts()

0.0     3661387
14.0    1265657
13.0     323307
12.0       9871
Name: Class Label, dtype: int64

In [41]:
df = df1[df1['Class Label']== 13]

In [42]:
df = df.reset_index(inplace = False)

In [43]:
df

,index,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
0,9871,1499446500.020481000,160.17.5.1,192.168.10.15,443,55200,52697,223506,001e4fd4ca2800c1b114eb310800,45000028dd8c4000f1063c79a0110501c0a80a0f,01bbd7a02089e8f2934ffbc6501081484cd40000,000000000000,fwd,1.0,13.0
1,9872,1499446500.023181000,160.17.5.1,192.168.10.15,443,55200,52697,223506,001e4fd4ca2800c1b114eb310800,450006bfdd984000f10635d6a0110501c0a80a0f,01bbd7a02089e8f2934ffbc650188148767b0000,1703030692a748cbe5a434db6d7825501a47f560ccdc33...,fwd,1.0,13.0
2,9873,1499446500.023234000,192.168.10.15,160.17.5.1,55200,443,52697,28717,00c1b114eb31001e4fd4ca280800,4500002844d0400080064636c0a80a0fa0110501,d7a001bb934ffbc62089ef895010faf0cc940000,000000000000,bwd,1.0,13.0
3,9874,1499446500.023745000,192.168.10.15,160.17.5.1,55200,443,52697,28717,00c1b114eb31001e4fd4ca280800,4500048844d14000800641d5c0a80a0fa0110501,d7a001bb934ffbc62089ef895018faf0fb790000,170303045b000000000000002676f1d6626a0da6d0d9d7...,bwd,1.0,13.0
4,9875,1499446500.026953000,160.17.5.1,192.168.10.15,443,55199,52696,285624,001e4fd4ca2800c1b114eb310800,4500002807bf4000f1061247a0110501c0a80a0f,01bbd79f0bd925f5bf461c365010807ed8e60000,000000000000,fwd,1.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323302,333173,1499448548.966478000,192.168.10.16,52.84.145.148,39586,443,60261,337,00c1b114eb310023ae9b8abf0800,45000034996e4000400610b5c0a80a1034549194,9aa201bb6b58302c1d24ac578010013190c60000010108...,00,bwd,1.0,13.0
323303,333174,1499448548.977391000,52.84.145.148,192.168.10.16,443,39592,60264,3453,0023ae9b8abf00c1b114eb310800,450001365c574000f50697c934549194c0a80a10,01bb9aa80ff461460498d9328018007642960000010108...,16030300ca040000c600002a3000c09fb3c529b3c60995...,fwd,1.0,13.0
323304,333175,1499448548.981589000,52.84.145.148,192.168.10.16,443,39590,60263,3453,0023ae9b8abf00c1b114eb310800,4500013682bf4000f506716134549194c0a80a10,01bb9aa695b5e60afe9591dc80180076328f0000010108...,16030300ca040000c600002a3000c09fb3c529b3c60995...,fwd,1.0,13.0
323305,333176,1499448548.982480000,192.168.10.16,52.84.145.148,39584,443,60260,337,00c1b114eb310023ae9b8abf0800,45000034161f400040069404c0a80a1034549194,9aa001bb8effa2fb30d3e2628010013147d80000010108...,00,bwd,1.0,13.0


In [44]:
ip_header_lst = []

for i in range(len(df)):
    x = df['IP Header'][i]
    byte_array = bytes.fromhex(x)
    byte_lst = list(byte_array)
    if (len(byte_lst) < 20):
            output = np.pad(byte_lst, (0, 20-len(byte_lst)), 'constant')
    else:
            output = np.array(byte_lst[0:20].copy())
            
    output = np.delete(output, [0, 1, 9, 12, 13, 14, 15, 16, 17, 18, 19])
    
    output = np.abs(output.astype(float))/255
    ip_header_lst.append(output)
    
df['IP Header Array'] = ip_header_lst

In [45]:
tcp_header_lst = []

for i in range(len(df)):
    x = df['TCP Header'][i]
    byte_array = bytes.fromhex(x)
    byte_lst = list(byte_array)
    if (len(byte_lst) < 20):
            output = np.pad(byte_lst, (0, 20-len(byte_lst)), 'constant')
    else:
            output = np.array(byte_lst[0:20].copy())
            
    output = np.delete(output, [0, 1, 2, 3])
    output = np.abs(output.astype(float))/255
    
    tcp_header_lst.append(output)
    
df['TCP Header Array'] = tcp_header_lst

In [46]:
df['pkt_header_values'] = df.apply(lambda x : (np.append(x['IP Header Array'], x['TCP Header Array'])), axis=1)

In [47]:
new_cols = ['Source IP', 'Dst IP', 'Source Port', 'Dst Port','Epoch Time','Stream Index', 'Sequence Number', 'Direction', 'pkt_header_values']
df = df[new_cols]

In [48]:
df

,Source IP,Dst IP,Source Port,Dst Port,Epoch Time,Stream Index,Sequence Number,Direction,pkt_header_values
0,160.17.5.1,192.168.10.15,443,55200,1499446500.020481000,52697,223506,fwd,"[0.0, 0.1568627450980392, 0.8666666666666667, ..."
1,160.17.5.1,192.168.10.15,443,55200,1499446500.023181000,52697,223506,fwd,"[0.023529411764705882, 0.7490196078431373, 0.8..."
2,192.168.10.15,160.17.5.1,55200,443,1499446500.023234000,52697,28717,bwd,"[0.0, 0.1568627450980392, 0.26666666666666666,..."
3,192.168.10.15,160.17.5.1,55200,443,1499446500.023745000,52697,28717,bwd,"[0.01568627450980392, 0.5333333333333333, 0.26..."
4,160.17.5.1,192.168.10.15,443,55199,1499446500.026953000,52696,285624,fwd,"[0.0, 0.1568627450980392, 0.027450980392156862..."
...,...,...,...,...,...,...,...,...,...
323302,192.168.10.16,52.84.145.148,39586,443,1499448548.966478000,60261,337,bwd,"[0.0, 0.20392156862745098, 0.6, 0.431372549019..."
323303,52.84.145.148,192.168.10.16,443,39592,1499448548.977391000,60264,3453,fwd,"[0.00392156862745098, 0.21176470588235294, 0.3..."
323304,52.84.145.148,192.168.10.16,443,39590,1499448548.981589000,60263,3453,fwd,"[0.00392156862745098, 0.21176470588235294, 0.5..."
323305,192.168.10.16,52.84.145.148,39584,443,1499448548.982480000,60260,337,bwd,"[0.0, 0.20392156862745098, 0.08627450980392157..."


In [49]:
streams_lst = list(df['Stream Index'].unique())

In [50]:
len(streams_lst)

7674

In [51]:
# 70% of benign data for train, 30% for test benign

In [52]:
#random.seed(0)
#mlist = random.sample(streams_lst, 61492)

In [53]:
#len(imlist)

In [54]:
#df_train = df.loc[df['Stream Index'].isin(imlist)]
#df_train = df_train.reset_index(inplace = False)

In [55]:
#df_test = df.loc[~df['Stream Index'].isin(imlist)]
#df_test = df_test.reset_index(inplace = False)

In [56]:
#streams_lst1 = list(df_test['Stream Index'].unique())

In [57]:
#len(streams_lst1)

In [58]:
df

,Source IP,Dst IP,Source Port,Dst Port,Epoch Time,Stream Index,Sequence Number,Direction,pkt_header_values
0,160.17.5.1,192.168.10.15,443,55200,1499446500.020481000,52697,223506,fwd,"[0.0, 0.1568627450980392, 0.8666666666666667, ..."
1,160.17.5.1,192.168.10.15,443,55200,1499446500.023181000,52697,223506,fwd,"[0.023529411764705882, 0.7490196078431373, 0.8..."
2,192.168.10.15,160.17.5.1,55200,443,1499446500.023234000,52697,28717,bwd,"[0.0, 0.1568627450980392, 0.26666666666666666,..."
3,192.168.10.15,160.17.5.1,55200,443,1499446500.023745000,52697,28717,bwd,"[0.01568627450980392, 0.5333333333333333, 0.26..."
4,160.17.5.1,192.168.10.15,443,55199,1499446500.026953000,52696,285624,fwd,"[0.0, 0.1568627450980392, 0.027450980392156862..."
...,...,...,...,...,...,...,...,...,...
323302,192.168.10.16,52.84.145.148,39586,443,1499448548.966478000,60261,337,bwd,"[0.0, 0.20392156862745098, 0.6, 0.431372549019..."
323303,52.84.145.148,192.168.10.16,443,39592,1499448548.977391000,60264,3453,fwd,"[0.00392156862745098, 0.21176470588235294, 0.3..."
323304,52.84.145.148,192.168.10.16,443,39590,1499448548.981589000,60263,3453,fwd,"[0.00392156862745098, 0.21176470588235294, 0.5..."
323305,192.168.10.16,52.84.145.148,39584,443,1499448548.982480000,60260,337,bwd,"[0.0, 0.20392156862745098, 0.08627450980392157..."


In [59]:
streams_lst = list(df['Stream Index'].unique())
print('Total number of flows in class:', len(streams_lst))    
random.shuffle(streams_lst)

# Directory path where to save the CSV files
directory_path = 'Friday_portscan_all/'

for PKT in range(2, 16):
    
    file_name = f'friday_portscan_{PKT}pkt.csv'
    file_path = directory_path + file_name
    

    data = []
    num_pkt = PKT
    pkt_val_pad = [0] * 25


    for j in range(len(streams_lst)):

        df_stream = df[df['Stream Index'] == streams_lst[j]]


        df_stream = df_stream.sort_values(by = 'Epoch Time')
        df_stream = df_stream.reset_index(drop=True)


        if len(df_stream) >= num_pkt:

            src_node = (int(netaddr.IPAddress(df_stream['Source IP'][0])), int(df_stream['Source Port'][0]))
            dst_node = (int(netaddr.IPAddress(df_stream['Dst IP'][0])), int(df_stream['Dst Port'][0]))
            stream_no = df_stream['Stream Index'][0]

            pkt_val_seq = []

            for k in range(0, num_pkt):

                pkt_val = []
                if df_stream['Direction'][k] == 'fwd':
                    pkt_val = [*df_stream['pkt_header_values'][k], *pkt_val_pad]
                else:
                    pkt_val = [*pkt_val_pad, *df_stream['pkt_header_values'][k]]

                pkt_val_seq.extend(pkt_val)

            data.append([src_node, dst_node, stream_no, pkt_val_seq])

    df1 = pd.DataFrame(data, columns= ['src_node', 'dst_node', 'stream_number', 'pkt_vals'])
    
    df1.to_csv(file_path, index=False)

Total number of flows in class: 7674
